- Aluno: Thiago Martin Poppe
- Matrícula: 2017014324

# Imports utilizados

In [1]:
import io
import sqlite3
import pandas as pd

# Estabelecendo a conexão e criando um cursor

In [2]:
conn = sqlite3.connect('tp1.db')
cursor = conn.cursor()

# Abrindo o arquivo .sql

In [3]:
f = io.open('despesas_publicas_tp1.sql', 'r')
sql = f.read()
cursor.executescript(sql)

# Questão 01

- Liste todos os códigos e nomes de subfunções da função que possui o maior número de subfunções.

In [4]:
sql_code_01 = """
select COD_SUBFUNCAO, NOME_SUBFUNCAO
from DETALHAMENTO_FUNCAO natural join
     SUBFUNCAO natural join
     (select COD_FUNCAO, count(*) as QTD
      from DETALHAMENTO_FUNCAO
      group by COD_FUNCAO
      order by QTD DESC LIMIT 1)
"""

df = pd.read_sql_query(sql_code_01, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (7, 2)


,COD_SUBFUNCAO,NOME_SUBFUNCAO
0,125,Normatização e fiscalização
1,126,Tecnologia da informação
2,131,Comunicação social
3,244,Assistência comunitária
4,306,Alimentação e nutrição
5,365,Educação infantil
6,511,Saneamento básico rural


# Questão 02

- Liste o valor total de despesas píblicas por função (nome) e subfunção (nome), apenas nos casos em que esse valor total excedeu 200 mil reais.

In [5]:
sql_code_02 = """
select NOME_FUNCAO, NOME_SUBFUNCAO, sum(VALOR_DESPESA) as VALOR_TOTAL
from FUNCAO natural join 
     DETALHAMENTO_FUNCAO natural join
     SUBFUNCAO natural join
     DESPESA_PUBLICA
group by NOME_FUNCAO, NOME_SUBFUNCAO
having VALOR_TOTAL > 200000
"""

df = pd.read_sql_query(sql_code_02, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (2, 3)


,NOME_FUNCAO,NOME_SUBFUNCAO,VALOR_TOTAL
0,Assistência social,Assistência comunitária,227883.77
1,Educação,Educação básica,334262.06


# Questão 03

- Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [6]:
sql_code_03 = """
select NOME_ORGAO, sum(VALOR_DESPESA) as VALOR_TOTAL
from ORGAO natural join
     DESPESA_PUBLICA
where ANO_DESPESA = 2018
group by NOME_ORGAO
order by VALOR_TOTAL desc
"""

df = pd.read_sql_query(sql_code_03, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (14, 2)


,NOME_ORGAO,VALOR_TOTAL
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,"Ministério da Ciência, Tecnologia, Inovações",0.00
6,Ministério da Fazenda,0.00
7,Ministério da Integração Nacional,0.00
8,Ministério da Justiça e Segurança Pública,0.00
9,Ministério das Relações Exteriores,0.00


# Questão 04

- Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Fazenda no primeiro semestre de 2018.

In [7]:
sql_code_04 = """
select MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, NOME_PROGRAMA_GOVERNO, VALOR_DESPESA
from ORGAO natural join
     DESPESA_PUBLICA natural join
     PROGRAMA_GOVERNO
where NOME_ORGAO = "Ministério da Fazenda" and MES_DESPESA <= 6 and ANO_DESPESA = 2018
"""

df = pd.read_sql_query(sql_code_04, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (7, 5)


,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,4,2018,Reserva de Contingência,GESTAO AEROPORTUARIA,0
1,1,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
2,2,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
3,3,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
4,4,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
5,5,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
6,6,2018,Reserva de Contingência,BRASIL SEM MISERIA,0


# Questão 05

- Liste o código e nome dos programas de governo que possuíram os 3 maiores valores de despesas totais na função de educação durante o ano de 2018.

In [8]:
sql_code_05 = """
select COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO
from PROGRAMA_GOVERNO natural join
     DESPESA_PUBLICA natural join
     DETALHAMENTO_FUNCAO natural join
     FUNCAO
where ANO_DESPESA = 2018 and NOME_FUNCAO = "Educação"
order by VALOR_DESPESA desc limit 3
"""

df = pd.read_sql_query(sql_code_05, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (3, 2)


,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,11,ENEM
1,14,PRONATEC
2,14,PRONATEC


# Questão 06

- Liste todos os nomes de funções e subfunções das despesas públicas do terceiro trimestre de 2018 e seus respectivos valores totais, apenas para o programa MAIS MEDICOS.

In [9]:
sql_code_06 = """
select NOME_FUNCAO, NOME_SUBFUNCAO, sum(VALOR_DESPESA) as VALOR_TOTAL
from FUNCAO natural join
     DETALHAMENTO_FUNCAO natural join
     SUBFUNCAO natural join
     DESPESA_PUBLICA natural join
     PROGRAMA_GOVERNO
where ANO_DESPESA = 2018 and (MES_DESPESA >= 7 and MES_DESPESA <= 9) and NOME_PROGRAMA_GOVERNO = "MAIS MEDICOS"
group by NOME_FUNCAO, NOME_SUBFUNCAO
"""

df = pd.read_sql_query(sql_code_06, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (4, 3)


,NOME_FUNCAO,NOME_SUBFUNCAO,VALOR_TOTAL
0,Defesa nacional,Administração geral,0
1,Educação,Assistência hospitalar e ambulatorial,0
2,Educação,Ensino superior,2400
3,Saúde,Atenção básica,11000


# Questão 12

In [10]:
sql_code_12 = """
select NOME_FUNCAO, NOME_SUBFUNCAO
from FUNCAO natural join
     DETALHAMENTO_FUNCAO natural join
     SUBFUNCAO
"""

df = pd.read_sql_query(sql_code_12, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (40, 2)


,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


# Questão 13

In [11]:
sql_code_13 = """
select MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO
from PROGRAMA_GOVERNO natural join
     (select *
      from DESPESA_PUBLICA
      where MODALIDADE_DESPESA = 'Reserva de Contingência')
"""

df = pd.read_sql_query(sql_code_13, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (676, 3)


,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.0,BRASIL SEM MISERIA
1,Reserva de Contingência,0.0,BRASIL SEM MISERIA
2,Reserva de Contingência,0.0,BRASIL SEM MISERIA
3,Reserva de Contingência,0.0,BRASIL SEM MISERIA
4,Reserva de Contingência,0.0,BRASIL SEM MISERIA
5,Reserva de Contingência,0.0,BRASIL SEM MISERIA
6,Reserva de Contingência,0.0,BRASIL SEM MISERIA
7,Reserva de Contingência,0.0,BRASIL SEM MISERIA
8,Reserva de Contingência,0.0,VIOLENCIA NO TRANSITO
9,Reserva de Contingência,0.0,VIOLENCIA NO TRANSITO


# Questão 14

In [12]:
sql_code_14 = """
select VALOR_DESPESA, NOME_ORGAO
from DESPESA_PUBLICA natural join
     (select *
      from ORGAO
      where NOME_ORGAO = "Ministério da Fazenda")
"""

df = pd.read_sql_query(sql_code_14, conn)
print("Tamanho da tabela:", df.shape)
df

Tamanho da tabela: (13, 2)


,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


# Questão 15

In [13]:
sql_code_15 = """
select NOME_PROGRAMA_GOVERNO, VALOR_DESPESA
from DESPESA_PUBLICA natural join
     (select *
      from ORGAO
      where NOME_ORGAO = "Ministério da Saúde") natural join
      PROGRAMA_GOVERNO
"""

df = pd.read_sql_query(sql_code_15, conn)
print('Tamanho da tabela:', df.shape)
df

Tamanho da tabela: (219, 2)


,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.0
1,VIVER SEM LIMITE,0.0
2,VIVER SEM LIMITE,0.0
3,VIVER SEM LIMITE,0.0
4,VIVER SEM LIMITE,0.0
5,VIVER SEM LIMITE,0.0
6,VIVER SEM LIMITE,0.0
7,VIVER SEM LIMITE,0.0
8,VIVER SEM LIMITE,0.0
9,VIVER SEM LIMITE,0.0


In [14]:
cursor.close()